In [1]:
def print_section(text):
    print('##############################################')
    print('###  ', text)
    print('##############################################')


print_section('Start imports')

import os
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))

import tqdm
import numpy as np
import pickle
import pandas as pd
import matplotlib.cm as cm
import matplotlib.axes as am
import matplotlib.pyplot as plt
%matplotlib widget

import dimod
import dwave
import dwave.system
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
import dwave_networkx as dnx
import minorminer

from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import io as particles_io
import src.leap_funcs.qubo.q_matrix as q_matrix

from src import leap_funcs as leap_funcs
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import parameterstudy

from src import h5py_funcs
from src.h5py_funcs import inspections, discoveries, init_custom_getstates, io, parameterstudy_using_info_file

print_section('Finished imports')


##############################################
###   Start imports
##############################################
no cupy available, imported numpy as usual
Custom getstate functions for dwave.cloud.config.models.ClientConfig, dwave.cloud.client.qpu.Client, dwave.cloud.solver.StructuredSolver, dwave.system.samplers.dwave_sampler.DWaveSampler, dwave.system.composites.embedding.FixedEmbeddingComposite have been initialized.
##############################################
###   Finished imports
##############################################


In [2]:
import io
print('io.DEFAULT_BUFFER_SIZE', io.DEFAULT_BUFFER_SIZE)

io.DEFAULT_BUFFER_SIZE 8192


In [3]:
print_section('Gather file and folder paths - Start')
info_file_name = r'study_params_small.h5'
#study_name = 'sub_2'
#study_name = 'sub_3'
#study_name = 'sub_4_2'
#study_name = 'sub_5'
#study_name = 'sub_5_1_1'
#study_name = 'sub_5_2'
#study_name = 'sub_5_3'
#study_name = 'sub_5_3_2'
#study_name = 'sub_7_1_2'
study_name = 'sub_7_1'

study_subs = ['', '_2', '_3', '_4', '_5']
#study_subs = ['']

if study_name == 'sub_1':
    ...
elif study_name == 'sub_2':
    info_name = r'study_params_small'
else:
    info_name = r'study_params_' + study_name[:3] + study_name[4:]
#info_name = r'study_params_sub7_1_5'

info_file_name = [info_name + sub + '.h5' for sub in study_subs]


study_folder_path = [pathlib.Path.cwd().joinpath('01_out',study_name+sub) for sub in study_subs]
print('study_folder_path:', study_folder_path)

info_file_name_path = [pathlib.Path.joinpath(sfp, ifn) for sfp, ifn in zip(study_folder_path, info_file_name)]
print('info_file_name_path:', info_file_name_path)
samples_folder_name = r'samples'
samples_folder_name_path = [pathlib.Path.joinpath(sfp, samples_folder_name) for sfp in study_folder_path]
print('samples_folder_name_path:', samples_folder_name_path)
print(str(info_file_name_path[0])[-3:])
assert np.array([i.exists() for i in info_file_name_path]).all(), f'Info file does not exist. Check info_file_name_path ({info_file_name_path})'
print(info_file_name_path)
print_section('Gather file and folder paths - Finished')

##############################################
###   Gather file and folder paths - Start
##############################################
study_folder_path: [PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1'), PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_2'), PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_3'), PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_4'), PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_5')]
info_file_name_path: [PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1/study_

In [4]:
print_section('Read info files - Start')
dict_infos_read = {i:h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=ifp, infoset_name = 'info')  for i,ifp in enumerate(info_file_name_path)}
#dddddd = dict_infos_read.copy()

dict_info_read = {}
#for key in reversed(dict_infos_read.keys()): #reversed to make sure info from initial study_file is contained as a|=b priorizes b
#    dict_info_read = dict_info_read | dddddd[key]

for i,ifp in enumerate(info_file_name_path):
    dict_info_read.update(h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=ifp, infoset_name = 'info'))

print_section('Read info files - Finished')

#dict_info_read


#orig:
#dict_info_read = h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=info_file_name_path, infoset_name = 'info')
#dict_info_read.keys()

##########
#
# The following test fails, because comparing dicts throws an error (below) when arrays are encountered, thats why it is left to the user to make sure the subs are compatible with the main study.
# "The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()" 
#
#########
#if len(info_file_name_path)>1:
#    for i in range(len(info_file_name_path)):
#        for key, value in dict_infos_read[0].items():
#            if key=='time_history': continue
#            else:
#                print(key)
#                if isinstance(value, np.ndarray):
#                    assert (value == dict_infos_read[i][key]).all(), print(value, dict_infos_read[i][key])
#                else:
#                    try: value == dict_infos_read[i][key]
#                    except Exception as e:
#                        print(e)
#                        print(value)
#                        print(dict_infos_read[i][key])

##############################################
###   Read info files - Start
##############################################
##############################################
###   Read info files - Finished
##############################################


In [6]:
print_section('Check read info files for unfinished runs - Start')
#print(dict_infos_read[0]['time_history'])
for i in range(len(dict_infos_read)):
    _tmp = []
    for key, value in dict_infos_read[i]['time_history'].items():
        if key == 'attrs':
            continue
        else:
            if not value['attrs']['finished']:
                _tmp.append(key)
    print('unfinished param sets:', len(_tmp), _tmp)
print('done with separate dicts, now go with merged dict')
_tmp = []
for key, value in dict_info_read['time_history'].items():
    if key == 'attrs':
        continue
    else:
        if not value['attrs']['finished']:
            _tmp.append(key)
print('unfinished param sets:', len(_tmp), _tmp)
del _tmp
print_section('Check read info files for unfinished runs - Finished')

##############################################
###   Check read info files for unfinished runs - Start
##############################################
unfinished param sets: 2181 ['zz_7909475811', 'zz_6773356991', 'zz_0671159834', 'zz_5809408420', 'zz_7182836644', 'zz_8100166375', 'zz_9594809908', 'zz_3401307320', 'zz_6849217208', 'zz_5574212566', 'zz_1974892964', 'zz_5951710574', 'zz_5879624070', 'zz_9537422756', 'zz_6489894880', 'zz_6913481782', 'zz_0384749410', 'zz_8962614601', 'zz_5437862409', 'zz_7058921096', 'zz_5008895297', 'zz_2994764253', 'zz_6613610488', 'zz_2099744399', 'zz_9741146835', 'zz_0416300949', 'zz_1513896682', 'zz_8533890822', 'zz_3684405374', 'zz_6840361363', 'zz_3769805295', 'zz_5849208605', 'zz_0489488901', 'zz_2937947912', 'zz_7860427402', 'zz_2026379471', 'zz_7292859121', 'zz_5008414655', 'zz_3425778357', 'zz_4008724035', 'zz_6890526992', 'zz_7819920361', 'zz_5693935054', 'zz_9674685362', 'zz_1776832378', 'zz_9073379090', 'zz_8416042242', 'zz_0276936179', 'zz_7

In [7]:
list_studies = list(dict_info_read['time_history'].keys())
list_studies.remove('attrs')
print('list of all studies:', len(list_studies), list_studies)

list of all studies: 2816 ['zz_8097006340', 'zz_3271526003', 'zz_5771100733', 'zz_6926620307', 'zz_0035448496', 'zz_5187077316', 'zz_0283319908', 'zz_7078527454', 'zz_1607154550', 'zz_0718801486', 'zz_4880416337', 'zz_3199431448', 'zz_6141606936', 'zz_3686594361', 'zz_3751853524', 'zz_7074557852', 'zz_4896149692', 'zz_6859867594', 'zz_6431177729', 'zz_3692460835', 'zz_9344025413', 'zz_8669389062', 'zz_1063149707', 'zz_3365147581', 'zz_2666922764', 'zz_7772712132', 'zz_2728652188', 'zz_3246246724', 'zz_8085842554', 'zz_4199855208', 'zz_2188248880', 'zz_3078711298', 'zz_9052476602', 'zz_9127208387', 'zz_4198252939', 'zz_2169158755', 'zz_0335574155', 'zz_3626572964', 'zz_8480470983', 'zz_6816191017', 'zz_9801759557', 'zz_8144652514', 'zz_2948642163', 'zz_2314053580', 'zz_9045097699', 'zz_4138906829', 'zz_4607778612', 'zz_1900500435', 'zz_4989572363', 'zz_7252953223', 'zz_8371351367', 'zz_8058220185', 'zz_3049613848', 'zz_0731748568', 'zz_6339681232', 'zz_1154740184', 'zz_6380713589', 'zz_

In [8]:
#print(dict_info_read['time_history']['zz_7293366434']['data'][2].decode('utf-8')[:4] == '2024')
#print(dict_info_read['time_history']['zz_7293366434']['attrs']['finished'])

In [9]:
print_section('Check read info files for unfinished runs again - Start')

for stud in list_studies:
    if (not dict_info_read['time_history'][stud]['attrs']['finished'])\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] in ('2024', '2025')):
        print('here 1', stud)
    elif dict_info_read['time_history'][stud]['attrs']['finished']\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] not in ('2024', '2025')):
        print('here 1_2', stud)
    elif dict_info_read['time_history'][stud]['attrs']['finished']\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] in ('2024', '2025')):
        #print('here 2')
        continue
    else:
        print('here else', stud)
        for key, value in dict_infos_read.items():
            if '2024' == value['time_history'][stud]['data'][2].decode('utf-8')[:4]:
                dict_info_read['time_history'][stud]['data'][1] = (value['time_history'][stud]['data'][1]+'.'.encode('utf-8'))[:-1]
                dict_info_read['time_history'][stud]['data'][2] = (value['time_history'][stud]['data'][2]+'.'.encode('utf-8'))[:-1]
                dict_info_read['time_history'][stud]['attrs']['finished'] = True
print('if this is the only line printed, then all studies in the info file are finished')
print_section('Check read info files for unfinished runs again - Finished')

##############################################
###   Check read info files for unfinished runs again - Start
##############################################
if this is the only line printed, then all studies in the info file are finished
##############################################
###   Check read info files for unfinished runs again - Finished
##############################################


In [10]:
print_section('Check estimated runtime - Start')
#print(dict_info_read['study'])
study_read = dict_info_read['study']['data']
study_read.dtype.names
est_runts = study_read['sets']['estimated_runtime'] * 10 # 10 runs per set
cum_est_runts = np.sum(est_runts)
#print(est_runts)
print('cummulative estimated runtime for 10000 samples each is [h]:', cum_est_runts/3600)
del study_read, est_runts, cum_est_runts
print_section('Check estimated runtime - Finished')


##############################################
###   Check estimated runtime - Start
##############################################
cummulative estimated runtime for 10000 samples each is [h]: 12.74252714801899
##############################################
###   Check estimated runtime - Finished
##############################################


In [11]:
print_section('Extract run identifiers and separate into un-/finished - Start')
array_identifiers, started_psets, finished_psets = h5py_funcs.inspections.extract_identifiers(dict_info_read = dict_info_read)

print('merged dict:')
print('  started_psets:', started_psets)
print('  finished_psets:', finished_psets)
print('  {p1}/{p2} p_sets have been started, \n {p3}/{p2} p_sets have been finished'.format(p1=len(started_psets), p2=array_identifiers.shape[0], p3=len(finished_psets)))

list_array_identifiers = []; list_started_psets = []; list_finished_psets = []
appenders = [list_array_identifiers, list_started_psets, list_finished_psets]
for d in dict_infos_read.values():
    appendees = h5py_funcs.inspections.extract_identifiers(dict_info_read = d)
#    print(len(appendees[0]), len(appendees[1]), len(appendees[2]))
    assert (array_identifiers==appendees[0]).all()
    list_array_identifiers.append(appendees[0])
    if len(list_started_psets) > 0:
#        print('here if')
        #list_started_psets.append([ap for ap in appendees[1] if not ap in list_started_psets[-1]])
        #list_finished_psets.append([ap for ap in appendees[2] if not ap in list_finished_psets[-1]])
        list_started_psets.append([ap for ap in appendees[1] if not ap in [e for ll in list_started_psets for e in ll]])
        list_finished_psets.append([ap for ap in appendees[2] if not ap in [e for ll in list_finished_psets for e in ll]])
    else:
#        print('here else')
        list_started_psets.append(appendees[1])
        list_finished_psets.append(appendees[2])
print('from individual runs accumulated:')
print('  list_started_psets:', list_started_psets)
print('  list_finished_psets:', list_finished_psets)
print('  {p1}/{p2} p_sets have been started, \n {p3}/{p2} p_sets have been finished'.\
      format(p1=[len(l) for l in list_started_psets], p2=[len(l) for l in list_array_identifiers], p3=[len(l) for l in list_finished_psets]))
print_section('Extract run identifiers and separate into un-/finished - Finished')

##############################################
###   Extract run identifiers and separate into un-/finished - Start
##############################################
merged dict:
  started_psets: [b'zz_8097006340', b'zz_3271526003', b'zz_5771100733', b'zz_6926620307', b'zz_0035448496', b'zz_5187077316', b'zz_0283319908', b'zz_7078527454', b'zz_1607154550', b'zz_0718801486', b'zz_4880416337', b'zz_3199431448', b'zz_6141606936', b'zz_3686594361', b'zz_3751853524', b'zz_7074557852', b'zz_4896149692', b'zz_6859867594', b'zz_6431177729', b'zz_3692460835', b'zz_9344025413', b'zz_8669389062', b'zz_1063149707', b'zz_3365147581', b'zz_2666922764', b'zz_7772712132', b'zz_2728652188', b'zz_3246246724', b'zz_8085842554', b'zz_4199855208', b'zz_2188248880', b'zz_3078711298', b'zz_9052476602', b'zz_9127208387', b'zz_4198252939', b'zz_2169158755', b'zz_0335574155', b'zz_3626572964', b'zz_8480470983', b'zz_6816191017', b'zz_9801759557', b'zz_8144652514', b'zz_2948642163', b'zz_2314053580', b'zz_904509769

In [26]:
print_section('Define read from/write to params - Start')
#read_from = ['sample_data', 'pickle']
#write_to = ['pickle', 'None']
read_from = 'sample_data'
write_to = 'pickle'
print('  read_from:', read_from, ' , possibilities: \'sample_data\', \'pickle\'')
print('  write_to:', write_to, ' , possibilities: \'pickle\', \'None\'')
dict_for_df = {}
print_section('Define read from/write to params - Finished')

##############################################
###   Define read from/write to params - Start
##############################################
  read_from: sample_data  , possibilities: 'sample_data', 'pickle'
  write_to: pickle  , possibilities: 'pickle', 'None'
##############################################
###   Define read from/write to params - Finished
##############################################


In [13]:
if read_from == 'sample_data':
    print_section('Read from sample data - Start')

    for sfnp, ids in zip(samples_folder_name_path,list_finished_psets):
        # samples_folder_name_path is a list of all the submissions to be read. usually of length 1.
        print('  reading from', sfnp)
        _tmp_study_name = sfnp.parent.stem
        print('  file name', _tmp_study_name)
        dict_for_df |= h5py_funcs.inspections.read_answers_to_dict(\
            samples_folder_name_path=sfnp\
            , array_identifiers=np.array(ids))#[::25]
    print(dict_for_df.__sizeof__()) # size in memory in bytes
    print(len(dict_for_df)) # number of keys in dict
    if write_to == 'pickle':
        with open(pathlib.Path('01_out',f'dict_for_df_{_tmp_study_name}.txt'), 'wb') as f:
            pickle.dump(dict_for_df, f, protocol=5)
    del _tmp_study_name
    print_section('Read from sample data - Finished')


##############################################
###   Read from sample data - Start
##############################################
reading from /mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1/samples
file name sub_7_1
10/10 file zz_0718801486.h5 in dir: True  9.1s/9.1s, 1.099 it/s 
reading from /mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_2/samples
file name sub_7_1_2
10/10 file zz_5574212566.h5 in dir: True  7.4s/7.4s, 1.345 it/s 
reading from /mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_3/samples
file name sub_7_1_3
10/10 file zz_5121445422.h5 in dir: True  8.4s/8.4s, 1.19 it/s 
reading from /mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_4/samples
file name sub_7_1_4
10/10 file zz_9936768047

In [14]:
if read_from == 'pickle':
    print_section('Read from pickle - Start')
    with open(pathlib.Path('01_out',f'dict_for_df_{study_name}.txt'), 'rb') as f:
        dict_for_df = pickle.load(f)
    print_section('Read from pickle - Finished')

In [15]:
is_combine_pickled_dicts = False
if is_combine_pickled_dicts:
    print_section('Combine dicts read via Pickle - Start')
    in_dicts = ['01_out\\dict_for_df_sub_6_2.txt'\
               ,'01_out\\dict_for_df_sub_6_2_2.txt']
    out_dicts = ['01_out\\dict_for_df_sub_6_2_combined.txt']
    out_dict = {}
    for in_d in in_dicts:
        print(f'  read {in_d}')
        with open(in_d, 'rb') as f:
            out_dict |= pickle.load(f)
    print('  finished reading all in_dicts')
    for key, val in out_dict.items():
        sample_set_length = (len(val['custom']['sampleset'].keys()))
        if sample_set_length != 10:
            print(key, sample_set_length)
    print_section('Combine dicts read via Pickle - Finished')

In [16]:
if is_combine_pickled_dicts:
    print_section('Write combined dicts read via Pickle to file - Start')
    out_dicts = ['01_out\\dict_for_df_sub_5_3_combined.txt']
    with open(out_dicts[0], 'wb') as f:
        pickle.dump(out_dict, f, protocol=5)
    print_section('Write combined dicts read via Pickle to file - Finished')

In [17]:
print_section('Obtain exact/correct solution via simulated annealing - Start')
import ast
import dwave.samplers

num_particles = 5
qubo = dict_info_read['qubos'][f'{num_particles}_{num_particles}']

#sim_annealing_sample = dimod.samplers.ExactSolver().sample_qubo(
#    {ast.literal_eval(key): value['data'] for key, value in qubo.items()})
sim_annealing_sample = dwave.samplers.SimulatedAnnealingSampler().sample_qubo(
    {ast.literal_eval(key): value['data'] for key, value in qubo.items()},
    num_reads=10000)
sim_annealing_sample = sim_annealing_sample.aggregate() # accumulates number of occurences
#print(type(sim_annealing_sample))
exact_sol = sim_annealing_sample.record
exact_sol.sort(order='energy')
#print(exact_sol)
print_section('Obtain correct solution via simulated annealing - Finished')

##############################################
###   Obtain exact/correct solution via simulated annealing - Start
##############################################
##############################################
###   Obtain correct solution via simulated annealing - Finished
##############################################


In [18]:
print_section('Extract success dict from info, samples, and correct solution - Start')
n_samples_to_compare = 3
n_exact_sols_to_compare = 3
#dir_name_path_plots = './01_out/'

success_dict = h5py_funcs.inspections.extract_success_dict(\
    dict_for_df = dict_for_df\
    , exact_sols = exact_sol\
    , n_samples_to_compare = n_samples_to_compare\
    , n_exact_sols_to_compare = n_exact_sols_to_compare)


#for key in success_dict.keys():
#    print(key, success_dict[key])
print_section('Extract success dict from info, samples, and correct solution - Finished')

##############################################
###   Extract success dict from info, samples, and correct solution - Start
##############################################
##############################################
###   Extract success dict from info, samples, and correct solution - Finished
##############################################


In [19]:
print_section('Extract started parameter sets from success dict - Start')
started_sets, study_matched_started_ids = h5py_funcs.inspections.extract_started_sets_from_success_dict(\
    success_dict = success_dict\
    , dict_info_read = dict_info_read)
#print(started_sets)
#print(study_matched_started_ids)
#print(study_matched_started_ids.dtype)
print_section('Extract started parameter sets from success dict - Finished')

##############################################
###   Extract started parameter sets from success dict - Start
##############################################
##############################################
###   Extract started parameter sets from success dict - Finished
##############################################


In [20]:
print_section('Generate sampler array for plots - Start')
results_names = list(list(success_dict.values())[0].keys())
to_remove = ['is_found_best_per_run', 'num_subs_per_run', 'num_samples_per_run', 'num_samples_per_sub_per_run', 'num_matched_per_run'\
             , 'num_matched_per_sub_per_run', 'num_samples_matched_per_run', 'num_samples_matched_per_sub_per_run', 'submissions', 'num_samples_is_found_best_per_run']
for s in to_remove:
    results_names.remove(s)
#print(results_names)



sampler_array,results_names, ids = h5py_funcs.inspections.generate_sampler_array_for_plots(\
    success_dict = success_dict\
    , results_names = results_names\
    , started_sets = started_sets\
    , study_matched_started_ids = study_matched_started_ids\
    , n_samples_to_compare = n_samples_to_compare\
    , n_exact_sols_to_compare = n_exact_sols_to_compare)

#print(sampler_array.shape)
#print(results_names)
print_section('Generate sampler array for plots - Finished')

##############################################
###   Generate sampler array for plots - Start
##############################################
##############################################
###   Generate sampler array for plots - Finished
##############################################


In [22]:
#print_section('Generate df from info file - Start')
df1 = pd.DataFrame(study_matched_started_ids['identifiers'], columns=['identifiers'], dtype='str')
df2 = pd.DataFrame().from_records(study_matched_started_ids['sets'])
df = df1.join(df2)
#print(df.head().to_string())
print(df.columns)
print_section('Generate df from info file - Finished')

Index(['identifiers', 'annealing_time', 'programming_thermalization',
       'readout_thermalization', 'flux_drift_compensation', 'chain_strength',
       'anneal_offsets_1_qubits', 'anneal_offsets_2_qubits',
       'anneal_offsets_3_qubits', 't00', 't01', 't02', 't03', 't04', 't05',
       't06', 't07', 't08', 't09', 't10', 't11', 's00', 's01', 's02', 's03',
       's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11',
       'estimated_runtime'],
      dtype='object')
##############################################
###   Generate df from info file - Finished
##############################################


In [23]:
print_section('Generate df from success dict - Start')

#print(list(success_dict['zz_8097006340'].keys())
#print(success_dict['zz_8097006340'])

df_success_dict = pd.DataFrame.from_dict(data=success_dict, orient='index')
#print(df_success_dict.head().to_string())
df_success_dict = df_success_dict.reset_index(names='identifiers')
#print(df_success_dict.head().to_string())

print_section('Generate df from success dict - Finished')

##############################################
###   Generate df from success dict - Start
##############################################
##############################################
###   Generate df from success dict - Finished
##############################################


In [28]:
print_section('Merge dfs from info file and success dict - Start')
df_complete = df.merge(df_success_dict, on='identifiers')
print(df_complete.head().to_string())
print_section('Merge dfs from info file and success dict - Finished')

##############################################
###   Merge dfs from info file and success dict - Start
##############################################
     identifiers  annealing_time  programming_thermalization  readout_thermalization  flux_drift_compensation  chain_strength  anneal_offsets_1_qubits  anneal_offsets_2_qubits  anneal_offsets_3_qubits  t00       t01       t02       t03       t04       t05       t06       t07       t08       t09       t10       t11  s00       s01       s02       s03       s04       s05       s06       s07       s08       s09       s10  s11  estimated_runtime  is_found_best  num_runs_is_found_best  num_samples_is_found_best num_samples_is_found_best_per_run                                         is_found_best_per_run  num_samples  num_runs                num_subs_per_run                                           num_samples_per_run                                                                                                                               